In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. 문서 로딩
print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('./data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

# 2. 문서 분할
print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,         # 로컬 모델 최적화
    chunk_overlap=150,      # 중복 감소
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

# 3. 벡터화
print("==> 3. 벡터화 → 임베딩으로 변환")
embeddings = OllamaEmbeddings(
    model="qwen3:1.7b",
    base_url="http://localhost:11434"
)

# 4. FAISS 저장
print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 5. 검색기 생성
print("==> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 성능 고려해 4개만
)

# 6. LLM 초기화
print("==> 6. 생성 → LLM으로 답변 생성")
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_predict=1500
)

# 7. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 8. 질문 실행 예시
query = "이 문서에서 가장 중요한 개념은 무엇인가요?"
result = qa_chain({"query": query})

print("\n===> 최종 답변:")
print(result["result"])

==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  76개 청크 생성 완료
==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
==> 5. 검색 → 질문과 유사한 문서 찾기
==> 6. 생성 → LLM으로 답변 생성


C:\Users\hun\AppData\Local\Temp\ipykernel_33100\2676010252.py:60: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})



===> 최종 답변:
<think>
Okay, let's see. The user is asking about the most important concept in the provided document. The document is about debugging in BlueJ, right? Let me go through the content again.

The text starts by explaining how a breakpoint works when a program stops at a breakpoint. It mentions using the Step Into button to debug method calls. There's talk about the debugger window showing the current line in the editor and the steps to take. Then there's a section on variables, explaining that variables are automatically updated in the debugger. The note about the project not being compiled initially and the Compile button is also mentioned.

So, the key points here are about debugging techniques like breakpoints, stepping into methods, and variable inspection. The main concepts would be the use of breakpoints, stepping through code, and inspecting variables in the debugger. The user might be looking for the core debugging strategies in BlueJ. I need to make sure I capture t